In [2]:
import numpy as np
import os
import pandas as pd
import tarfile
import gzip
import os
import nltk 
import pandas as pd
from nltk.tokenize import word_tokenize   
from nltk.corpus import stopwords         
from nltk.stem import WordNetLemmatizer   
from nltk.stem import PorterStemmer      
import string
from tqdm import tqdm
from glob import glob
import re

In [43]:
import csv
for root,dirs,files in os.walk("./tweets"):
    for i in tqdm(range(len(files))):
        file = files[i]
        file_path = os.path.join(root,file)
        with open(file_path,'r',encoding='UTF-8') as csvfile:
            csv_reader = csv.reader(csvfile) 
            header = next(csv_reader)  
            for row in csv_reader:  
                if len(row)==3:
                    id = row[0]
                    created_at = row[1]
                    full_text = row[2]
                    with open("full_text.csv",'a',newline="", encoding= 'UTF-8') as csv_file:
                        csvwriter = csv.writer(csv_file)
                        csvwriter.writerow([id, created_at,full_text])

0it [00:00, ?it/s]
100%|██████████████████████████████████████████████████████████████████████████████| 6567/6567 [02:25<00:00, 45.29it/s]


In [24]:
id_text = pd.read_csv("data_processed.csv")

In [25]:
id_text

,id,text_processed
0,1587956423287508992,china beijing report new local symptomatic cor...
1,1587956409366466560,mean use chinese app e share personal data chi...
2,1587956402483503104,hat important thing important china china ukra...
3,1587956384473260034,never flourish would inside worlds large marke...
4,1587956376931774465,seem uk policy make circle still explore way p...
...,...,...
1136256,1591265116338548736,would constitution block republican meanwhile ...
1136257,1591265106717192192,hate troll see russia aggrieve ukraine call wa...
1136258,1591265105358245888,ukraine crowdfunding new fleet drone boat take...
1136259,1591265101746929664,would think celebrate retreat russia ohh wait ...


In [81]:
id_doc = dict(zip(data['id'],data["text_processed"]))
id_index = dict(zip(data['id'],data.index))

## inverted index

In [28]:
invert_dict={}
for i in range(len(data)):
    id = id_text['id'][i]
    text = id_text['text_processed'][i]
    if type(text)!=str:
        text = ''
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    for word in tokens:
        if word in invert_dict:
            invert_dict[word].append(id)
        else:
            invert_dict[word]=[id]

In [30]:
#write csv
import csv
with open("index.csv",'w',newline="", encoding= 'UTF-8') as csv_file:
    csvwriter = csv.writer(csv_file)
    csvwriter.writerow(["word","ID"])
    for key in invert_dict:
        csvwriter.writerow([key,invert_dict[key]])

In [133]:
#read csv
import csv
invert_dict={}
index=pd.read_csv("index.csv",header=None,encoding = "UTF-8")
for i in range(1,len(index[0])):
    w=[]
    l=index[1][i][1:-1].split(',')
    invert_dict[index[0][i]]=l

# Query

In [37]:
import tarfile
import gzip
import os
import nltk 
import pandas as pd
from nltk.tokenize import word_tokenize   
from nltk.corpus import stopwords         
from nltk.stem import WordNetLemmatizer   
from nltk.stem import PorterStemmer      
import string
from tqdm import tqdm
from glob import glob
import re
def text_process(text):
    res = []
    if text:
        text = text.strip('RT')
        at_pattern = re.compile("@[\S]*")
        url_pattern = re.compile("https:\/\/[\w./]*[\w/]")
        text = re.sub(at_pattern,'',text)
        text = re.sub(url_pattern,'',text)
        tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
        tokens = tokenizer.tokenize(text.lower())
        stop_words = set(stopwords.words('english'))
        stop_words.update(list(string.punctuation))
        filtered = [w for w in tokens if w not in stop_words]
        filtered = [w for w in tokens if not w.isdigit()]
        lemmatizaer = WordNetLemmatizer()
        for word, tag in nltk.pos_tag(filtered):
            if tag.startswith('NN'):
                word= lemmatizaer.lemmatize(word, pos='n')
            elif tag.startswith('VB'):
                word= lemmatizaer.lemmatize(word, pos='v')
            elif tag.startswith('JJ'):
                word= lemmatizaer.lemmatize(word, pos='a')
            elif tag.startswith('R'):
                word= lemmatizaer.lemmatize(word, pos='r')
            res.append(word)
    return res

In [38]:
query = "CHINA BEIJING CORONAVIRUS CASES 2022"
q = text_process(query)

In [39]:
q

['china', 'beijing', 'coronavirus', 'case']

# Boolean Query

In [66]:
def boolean_query(q):
    index=set(invert_dict[q[0]])
    for word in q:
        if(len(word)>1):
            index=index&set(invert_dict[word])
    index=list(index)
    index.sort()
    return index

In [67]:
index = boolean_query(q)

In [68]:
index

[1587956423287508992,
 1588142773349371906,
 1589406732773920768,
 1589406754231779330,
 1589407497634422785,
 1589421735958429696,
 1589812052046270466,
 1589908442042404866,
 1589909473841364995,
 1590131480541687808,
 1590131502179766272,
 1590131506797719553,
 1590131566923386881,
 1590389848300728321,
 1590492823107010560,
 1590492844615155715,
 1590492846137626625,
 1590492905285771267,
 1590494088255741953,
 1590601401385697280]

In [69]:
for doc in index:
    print(doc_dict[doc])

china beijing report new local symptomatic coronavirus case asymptomatic case nov
china covid universal resort shuts due beijing coronavirus case business
china beijing report new local symptomatic coronavirus case asymptomatic case nov
china beijing report new local symptomatic coronavirus case asymptomatic case nov
china beijing report new local symptomatic coronavirus case asymptomatic case nov
firstsquawk china beijing report new local symptomatic coronavirus case asymptomatic case nov
nstworld china capital beijing report symptomatic asymptomatic case compare symptomatic asymptomatic case previous day local government data show china coronavirus infection
china covid epicentre shift guangzhou outbreaks widen beijing november new coronavirus case surge guangzhou chinese city official data show tuesday global manufacturing hub become
beijing new coronavirus case surge guangzhou chinese city official data show tuesday global manufacturing hub become china late epicentre test city abi

# Sort by TF-IDF

In [49]:
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.feature_extraction.text import TfidfTransformer 
corpus = id_text['text_processed'].tolist()

In [50]:
for i in range(len(corpus)):
    if type(corpus[i])!=str:
        corpus[i]=''
    corpus[i].lower()

In [51]:
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import operator

## Create Vocabulary
vectorizer = CountVectorizer()  
X = vectorizer.fit_transform(corpus)  
vocabulary = vectorizer.get_feature_names_out() 
# Intializating the tfIdf model
tfidf = TfidfVectorizer(vocabulary=vocabulary)
# Fit the TfIdf model
tfidf.fit(corpus)
# Transform the TfIdf model
tfidf_tran=tfidf.transform(corpus)

In [43]:
tfidf_tran.shape

(1136261, 208629)

In [46]:
type(tfidf_tran)

scipy.sparse.csr.csr_matrix

In [48]:
from scipy import sparse
sparse.save_npz('tf_idf.npz', tfidf_tran)

In [ ]:
tfidf = sparse.load_npz('tf_idf.npz')

In [60]:
def gen_vector(tokens):
    Q = np.zeros((len(vocabulary)))   
    for token in tokens:
        try:
            ind = tfidf.vocabulary_[token]
            Q[ind]  += 1/len(tokens)
        except:
            pass     
    for token in tokens:
        try:
            ind = tfidf.vocabulary_[token]
            Q[ind]  = tfidf.idf_[ind]
        except:
            pass
    return Q

In [61]:
query_vector = gen_vector(q)

In [74]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial
    
#rank the documents in index
d_cosines = []
for doc in index:
    #get the tfidf of doc
    d = np.asarray(tfidf_tran.getrow(id_index[int(doc)]).todense())
    #compute cosine similarity
    cos_sim = cosine_similarity(query_vector.reshape(1, -1), d.reshape(1, -1))[0][0]
    d_cosines.append(cos_sim)   
out = np.array(d_cosines).argsort()[-10:][::-1]

[0.6107428986720824, 0.5356085059414503, 0.6107428986720824, 0.6107428986720824, 0.6107428986720824, 0.5406374664321026, 0.3725669053914238, 0.3402297410808934, 0.34580807625424526, 0.6107428986720824, 0.6107428986720824, 0.6107428986720824, 0.43649124679078805, 0.3615591548329079, 0.6107428986720824, 0.6107428986720824, 0.6107428986720824, 0.47116954449854526, 0.6042208667041988, 0.6042208667041988]


In [84]:
for i in range(10):
    print(index[out[i]])
    print(id_doc[index[out[i]]])

1590131480541687808
china beijing report new local symptomatic coronavirus case asymptomatic case nov
1590131506797719553
china beijing report new local symptomatic coronavirus case asymptomatic case nov
1589406732773920768
china beijing report new local symptomatic coronavirus case asymptomatic case nov
1589406754231779330
china beijing report new local symptomatic coronavirus case asymptomatic case nov
1589407497634422785
china beijing report new local symptomatic coronavirus case asymptomatic case nov
1590131502179766272
china beijing report new local symptomatic coronavirus case asymptomatic case nov
1587956423287508992
china beijing report new local symptomatic coronavirus case asymptomatic case nov
1590492823107010560
china beijing report new local symptomatic coronavirus case asymptomatic case nov
1590492844615155715
china beijing report new local symptomatic coronavirus case asymptomatic case nov
1590492846137626625
china beijing report new local symptomatic coronavirus case as

# search engine

In [3]:
import csv
import numpy as np
import os
import pandas as pd
import os
import nltk 
import pandas as pd
from nltk.tokenize import word_tokenize   
from nltk.corpus import stopwords         
from nltk.stem import WordNetLemmatizer   
from nltk.stem import PorterStemmer      
import string
from tqdm import tqdm
from glob import glob
import re
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import operator
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial

load data

In [4]:
full_text = pd.read_csv("full_text.csv",names=['ID', 'created_at','full_text'])
id_text = pd.read_csv("data_processed.csv")
id_doc = dict(zip(id_text['id'],id_text["text_processed"]))
id_index = dict(zip(id_text['id'],id_text.index))
invert_dict={}
index_csv=pd.read_csv("index.csv",header=None,encoding = "UTF-8")
for i in range(1,len(index_csv[0])):
    w=[]
    l=index_csv[1][i][1:-1].split(',')
    invert_dict[index_csv[0][i]]=l

calculate tf-idf

In [5]:
corpus = id_text['text_processed'].tolist()
for i in range(len(corpus)):
    if type(corpus[i])!=str:
        corpus[i]=''
    corpus[i].lower()

## Create Vocabulary
vectorizer = CountVectorizer()  
X = vectorizer.fit_transform(corpus)  
vocabulary = vectorizer.get_feature_names_out() 
# Intializating the tfIdf model
tfidf = TfidfVectorizer(vocabulary=vocabulary, lowercase = True)
# Fit the TfIdf model
tfidf.fit(corpus)
# Transform the TfIdf model
tfidf_tran=tfidf.transform(corpus)

D:\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1330: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


search

In [6]:
def text_process(text):
    res = []
    if text:
        text = text.strip('RT')
        at_pattern = re.compile("@[\S]*")
        url_pattern = re.compile("https:\/\/[\w./]*[\w/]")
        text = re.sub(at_pattern,'',text)
        text = re.sub(url_pattern,'',text)
        tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
        tokens = tokenizer.tokenize(text.lower())
        stop_words = set(stopwords.words('english'))
        stop_words.update(list(string.punctuation))
        filtered = [w for w in tokens if w not in stop_words]
        filtered = [w for w in tokens if w.isalpha()]
        lemmatizaer = WordNetLemmatizer()
        for word, tag in nltk.pos_tag(filtered):
            if tag.startswith('NN'):
                word= lemmatizaer.lemmatize(word, pos='n')
            elif tag.startswith('VB'):
                word= lemmatizaer.lemmatize(word, pos='v')
            elif tag.startswith('JJ'):
                word= lemmatizaer.lemmatize(word, pos='a')
            elif tag.startswith('R'):
                word= lemmatizaer.lemmatize(word, pos='r')
            res.append(word)
    return res

def boolean_query(q):
    for word in q:
        if(word in invert_dict):
            index=set(invert_dict[word])
            break
    for word in q:
        if(len(word)>1 and word in invert_dict):
            index=index&set(invert_dict[word])
    index=list(index)
    index.sort()
    return index

# generate tf-idf vector of query
def gen_vector(tokens):
    Q = np.zeros((len(vocabulary)))   
    for token in tokens:
        try:
            ind = tfidf.vocabulary_[token]
            Q[ind]  += 1/len(tokens)
        except:
            pass     
    for token in tokens:
        try:
            ind = tfidf.vocabulary_[token]
            Q[ind]  = tfidf.idf_[ind]
        except:
            pass
    return Q

In [7]:
def search_rank(query,k):
    q = text_process(query)
    query_vector = gen_vector(q)
    #boolean query
    index = boolean_query(q) 
    #rank the documents in index
    d_cosines = []
    for doc in index:
        #get the tfidf of doc
        d = np.asarray(tfidf_tran.getrow(id_index[int(doc)]).todense())
        #compute cosine similarity
        cos_sim = cosine_similarity(query_vector.reshape(1, -1), d.reshape(1, -1))[0][0]
        d_cosines.append(cos_sim)   
    out = np.array(d_cosines).argsort()[-k:][::-1]

    res = []
    for i in range(k):
        res.append(int(index[out[i]]))
    return res

In [12]:
res = search_rank("CHINA BEIJING CORONAVIRUS",10)

In [13]:
print(res)

[1590601401385697280, 1590494088255741953, 1588142773349371906, 1588172706180386816, 1590216955704885249, 1588412085695815680, 1588412082583736320, 1587956423287508992, 1589406732773920768, 1590131480541687808]


In [14]:
full_text[full_text['ID'].isin(res)]

,ID,created_at,full_text
0,1587956423287508992,Wed Nov 02 23:54:48 +0000 2022,CHINA'S BEIJING REPORTS 28 NEW LOCAL SYMPTOMAT...
28272,1588142773349371906,Thu Nov 03 12:15:18 +0000 2022,China Covid: Universal Resort shuts due to Bei...
34256,1588172706180386816,Thu Nov 03 14:14:14 +0000 2022,China Coronavirus Live: Why Beijing Locked Dow...
70779,1588412085695815680,Fri Nov 04 06:05:27 +0000 2022,Germany's leader and top CEOs have arrived in ...
70780,1588412082583736320,Fri Nov 04 06:05:26 +0000 2022,Germany's leader and top CEOs have arrived in ...
212001,1589406732773920768,Sun Nov 06 23:57:49 +0000 2022,CHINA'S BEIJING REPORTS 41 NEW LOCAL SYMPTOMAT...
333850,1590131480541687808,Tue Nov 08 23:57:42 +0000 2022,CHINA'S BEIJING REPORTS 32 NEW LOCAL SYMPTOMAT...
344726,1590216955704885249,Wed Nov 09 05:37:21 +0000 2022,China Coronavirus Live: Why Beijing Locked Dow...
384439,1590494088255741953,Wed Nov 09 23:58:35 +0000 2022,"Beijing reports 95 new coronavirus cases, low ..."
398115,1590601401385697280,Thu Nov 10 07:05:00 +0000 2022,"Beijing reports 95 new coronavirus cases, low ..."


In [11]:
print(full_text[full_text['ID'].isin(res)])

                         ID                      created_at  \
0       1587956423287508992  Wed Nov 02 23:54:48 +0000 2022   
211995  1589406754231779330  Sun Nov 06 23:57:54 +0000 2022   
212001  1589406732773920768  Sun Nov 06 23:57:49 +0000 2022   
212076  1589407497634422785  Mon Nov 07 00:00:51 +0000 2022   
333846  1590131506797719553  Tue Nov 08 23:57:49 +0000 2022   
333847  1590131502179766272  Tue Nov 08 23:57:47 +0000 2022   
333850  1590131480541687808  Tue Nov 08 23:57:42 +0000 2022   
384221  1590492846137626625  Wed Nov 09 23:53:39 +0000 2022   
384222  1590492844615155715  Wed Nov 09 23:53:38 +0000 2022   
384224  1590492823107010560  Wed Nov 09 23:53:33 +0000 2022   

                                                full_text  
0       CHINA'S BEIJING REPORTS 28 NEW LOCAL SYMPTOMAT...  
211995  CHINA'S BEIJING REPORTS 41 NEW LOCAL SYMPTOMAT...  
212001  CHINA'S BEIJING REPORTS 41 NEW LOCAL SYMPTOMAT...  
212076  CHINA'S BEIJING REPORTS 41 NEW LOCAL SYMPTOMAT...  
333846